Importy i utworzone funkcje


In [11]:
import random
import numpy as np

def determineWinner(qA, qB):
    if(qA == qB):
        return 'Noone'
    if(qA == 'SCISSORS' and qB == 'PAPER') or \
        (qA == 'PAPER' and qB == 'ROCK') or \
            (qA == 'ROCK' and qB == 'SCISSORS'):
        return 'A'
    return 'B'


def updateRanking(ranking, ranked, winner):
    if(winner == ranked):
        ranking['Won'] += 1
        return
    if(winner == 'Noone'):
        ranking['Draw'] += 1
        return
    ranking['Lost'] += 1


Algorytm "uczenia się": program uczy się na podstawie częstości występowania stanów użytkownika B, a dokładniej przy każdej iteracji zwiększa delikatnie prawdopodobieństwo przejścia do stanu wygrywającego z obecnym stanem B(dla każdego stanu w macierzy A). Wszystkie inne, sensowniejsze, deterministyczne rozwiązania niestety byłyby niezgodne z implementacją łańcuchów Markova. Jak widać, zysk takiego rozwiązania jest niewielki jeśli w ogóle obecny.

In [12]:
#self-'learning' algorithm
def updateTransMatrix(matrix, counters, q, learningFactor):
    rows = matrix.keys()
    for row in rows:
        i = 0
        for state in matrix[row]['next']:
            if (state == counters[q]):
                if(not matrix[row]['probabilities'][i]+learningFactor > 1):
                    matrix[row]['probabilities'][i] += learningFactor
                    round(matrix[row]['probabilities'][i], 2)
            else:
                if(not matrix[row]['probabilities'][i]-learningFactor/(len(rows)-1) < 0):
                    matrix[row]['probabilities'][i] -= learningFactor / \
                        (len(rows)-1)
                    round(matrix[row]['probabilities'][i], 2)
            i += 1

Implementacja, symulacja i wypisanie

In [20]:

states = ['SCISSORS', 'PAPER', 'ROCK']
# setting the default states with exemplary values
qA_0 = 'SCISSORS'
qB_0 = 'PAPER'

counters = {
    'PAPER': 'SCISSORS',
    'SCISSORS': 'ROCK',
    'ROCK': 'PAPER'
}

# initializing default transition 'matrixes' with exemplary values
transitionsA = {
    'SCISSORS': {'next': ['SCISSORS', 'PAPER', 'ROCK'], 'probabilities': [0.1, 0.3, 0.6]},
    'PAPER': {'next': ['SCISSORS', 'PAPER', 'ROCK'], 'probabilities': [0.3, 0.3, 0.4]},
    'ROCK': {'next': ['SCISSORS', 'PAPER', 'ROCK'], 'probabilities': [0.5, 0.3, 0.2]}
}
transitionsB = {
    'SCISSORS': {'next': ['SCISSORS', 'PAPER', 'ROCK'], 'probabilities': [0.35, 0.45, 0.2]},
    'PAPER': {'next': ['SCISSORS', 'PAPER', 'ROCK'], 'probabilities': [0.2, 0.05, 0.75]},
    'ROCK': {'next': ['SCISSORS', 'PAPER', 'ROCK'], 'probabilities': [0.3, 0.5, 0.2]}
}

for i in range(2):
    #two simulations, one with the self-learning, one without    
    aStats = {
        'Won': 0,
        'Draw': 0,
        'Lost': 0
    }

    # setting the simulation
    qA = qA_0
    qB = qB_0
    simulation_number = 50

    for j in range(simulation_number):
        # determining winner
        winner = determineWinner(qA, qB)
        updateRanking(aStats, 'A', winner)

        # iteration printout
        print('='*40)
        print('Game {}'.format(j))
        print('User A has: {} and User B has: {}'.format(qA, qB))
        print(winner+' wins!')

        # self-'learning'
        if(i==1):
            updateTransMatrix(transitionsA, counters, qB, 0.04)

        # progressing the chain
        qA = random.choices(transitionsA[qA]['next'],
                            transitionsA[qA]['probabilities'])[0]
        qB = random.choices(transitionsB[qB]['next'],
                            transitionsB[qB]['probabilities'])[0]

    print('='*40)
    print('User A:\r\nWon: {}%\r\nDraw: {}%\r\nLost: {}%'
            .format(aStats['Won']*100/simulation_number,
                    aStats['Draw']*100/simulation_number,
                aStats['Lost']*100/simulation_number))

    print('='*40)
    # converting to genuine matrixes for printing
    a = np.array([
        ['userA', 'SCISSORS', 'PAPER', 'ROCK'],
        ['SCISSORS', transitionsA['SCISSORS']['probabilities'][0], transitionsA['SCISSORS']
            ['probabilities'][1], transitionsA['SCISSORS']['probabilities'][2]],
        ['PAPER', transitionsA['PAPER']['probabilities'][0], transitionsA['PAPER']
            ['probabilities'][1], transitionsA['PAPER']['probabilities'][2]],
        ['ROCK', transitionsA['ROCK']['probabilities'][0], transitionsA['ROCK']
            ['probabilities'][1], transitionsA['ROCK']['probabilities'][2]]
    ])
    b = np.array([
        ['userB', 'SCISSORS', 'PAPER', 'ROCK'],
        ['SCISSORS', transitionsB['SCISSORS']['probabilities'][0], transitionsB['SCISSORS']
            ['probabilities'][1], transitionsB['SCISSORS']['probabilities'][2]],
        ['PAPER', transitionsB['PAPER']['probabilities'][0], transitionsB['PAPER']
            ['probabilities'][1], transitionsB['PAPER']['probabilities'][2]],
        ['ROCK', transitionsB['ROCK']['probabilities'][0], transitionsB['ROCK']
            ['probabilities'][1], transitionsB['ROCK']['probabilities'][2]]
    ])
    print(a)
    print('='*40)
    print(b)


Game 0
User A has: SCISSORS and User B has: PAPER
A wins!
Game 1
User A has: ROCK and User B has: ROCK
Noone wins!
Game 2
User A has: SCISSORS and User B has: SCISSORS
Noone wins!
Game 3
User A has: ROCK and User B has: PAPER
B wins!
Game 4
User A has: PAPER and User B has: ROCK
A wins!
Game 5
User A has: SCISSORS and User B has: SCISSORS
Noone wins!
Game 6
User A has: PAPER and User B has: SCISSORS
B wins!
Game 7
User A has: ROCK and User B has: PAPER
B wins!
Game 8
User A has: PAPER and User B has: SCISSORS
B wins!
Game 9
User A has: ROCK and User B has: PAPER
B wins!
Game 10
User A has: ROCK and User B has: PAPER
B wins!
Game 11
User A has: SCISSORS and User B has: ROCK
B wins!
Game 12
User A has: ROCK and User B has: PAPER
B wins!
Game 13
User A has: SCISSORS and User B has: ROCK
B wins!
Game 14
User A has: ROCK and User B has: ROCK
Noone wins!
Game 15
User A has: SCISSORS and User B has: PAPER
A wins!
Game 16
User A has: PAPER and User B has: SCISSORS
B wins!
Game 17
User A has: S